In [100]:
#run only on Jan, Apr, July and Oct
from datetime import date
month = date.today().strftime("%m")
month = int(month)
runningMonths = (1,4,7,10)
if month not in runningMonths:
    raise SystemExit("Stop right there!")
else:
    print('continue')

continue


In [1]:
# The code was removed by Watson Studio for sharing.

In [2]:
#send POST request
res = requests.post(url, headers=handle)
#print(res.text)

In [3]:
#parse output to retrieve token
token = eval(res.text)
token = token['Result']
#print(token)

In [132]:
# The code was removed by Watson Studio for sharing.

,quarter
0,19Q3
1,19Q4
2,20Q1


In [133]:
#determine which q to process
last_q = list(quarter_df['quarter'])[-1]
[year, quarter] = list(map(int,last_q.split("Q")))
if quarter == 4:
    year = year + 1
    quarter = 1
else:
     quarter = quarter + 1   
q = "Q".join(list(map(str,[year,quarter])))
q 

'20Q2'

In [134]:
##connect to API and download data (.json format)
#select quarter (19q3)
handle['Token'] = token
url = 'https://www.ura.gov.sg/uraDataService/invokeUraDS?service=PMI_Resi_Rental&refPeriod=' + q

In [135]:
#connect to API
res = requests.post(url, headers=handle)
json_out = res.text

In [136]:
#convert data to json object
import json
ura_json = json.loads(json_out)

In [137]:
#convert nested json to df
from pandas.io.json import json_normalize
ura_df = json_normalize(data=ura_json['Result'], record_path='rental',
                     meta=['project', 'street','x','y'], errors='ignore')


In [138]:
ura_df.shape

(20097, 11)

In [139]:
ura_df.head()

,areaSqft,areaSqm,district,leaseDate,noOfBedRoom,propertyType,rent,project,street,x,y
0,2300-2400,210-220,15,0620,3,Non-landed Properties,4650,ELLIOT AT THE EAST COAST,ELLIOT ROAD,38883.874192089998,32621.50999211
1,2300-2400,210-220,15,0620,3,Non-landed Properties,5600,ELLIOT AT THE EAST COAST,ELLIOT ROAD,38883.874192089998,32621.50999211
2,2500-2600,230-240,15,0420,4,Non-landed Properties,6200,ELLIOT AT THE EAST COAST,ELLIOT ROAD,38883.874192089998,32621.50999211
3,900-1000,90-100,15,0520,2,Non-landed Properties,3100,ELLIOT AT THE EAST COAST,ELLIOT ROAD,38883.874192089998,32621.50999211
4,2500-2600,230-240,15,0520,4,Non-landed Properties,5700,ELLIOT AT THE EAST COAST,ELLIOT ROAD,38883.874192089998,32621.50999211


In [140]:
#recalculate areaSqft to midpoint
import re
area = ura_df['areaSqft']
area = area.str.split("-")
for i in range(0, area.size):
    if len(area[i]) == 2:
        temp = list(map(int, area[i]))
        area[i] = sum(temp)/2
    else:
        temp = re.sub("[^0-9]","",area[i][0])
        area[i] = int(temp)
area.unique()

array([2350.0, 2550.0, 950.0, 1750.0, 2850.0, 1350.0, 1050.0, 2050.0,
       2750.0, 2250.0, 5750.0, 1250.0, 450.0, 8000, 850.0, 1450.0, 1650.0,
       2150.0, 3000, 2650.0, 2950.0, 2450.0, 3250.0, 550.0, 1150.0, 750.0,
       650.0, 1550.0, 4250.0, 1850.0, 3750.0, 350.0, 4750.0, 1950.0, 1000,
       7750.0, 6750.0, 5250.0, 6250.0, 7250.0, 250.0], dtype=object)

In [141]:
#calculate price per square feet (psf)
psf = ura_df['rent']/area
psf.head()

0    1.97872
1    2.38298
2    2.43137
3    3.26316
4    2.23529
dtype: object

In [142]:
#append psf column to ura_df
ura_df['psf'] = psf
ura_df['areaSqft'] = area
ura_df.head()

,areaSqft,areaSqm,district,leaseDate,noOfBedRoom,propertyType,rent,project,street,x,y,psf
0,2350,210-220,15,0620,3,Non-landed Properties,4650,ELLIOT AT THE EAST COAST,ELLIOT ROAD,38883.874192089998,32621.50999211,1.97872
1,2350,210-220,15,0620,3,Non-landed Properties,5600,ELLIOT AT THE EAST COAST,ELLIOT ROAD,38883.874192089998,32621.50999211,2.38298
2,2550,230-240,15,0420,4,Non-landed Properties,6200,ELLIOT AT THE EAST COAST,ELLIOT ROAD,38883.874192089998,32621.50999211,2.43137
3,950,90-100,15,0520,2,Non-landed Properties,3100,ELLIOT AT THE EAST COAST,ELLIOT ROAD,38883.874192089998,32621.50999211,3.26316
4,2550,230-240,15,0520,4,Non-landed Properties,5700,ELLIOT AT THE EAST COAST,ELLIOT ROAD,38883.874192089998,32621.50999211,2.23529


In [143]:
#reformat leaseDate to YYQN, where N is (1,2,3,4)
#define function to assign quarter
def assignQ(month):
    month = int(month)
    for i in range(1,5):
        if month <= i*3 and month > (i-1)*3:
            result = "Q" + str(i)
    return result

def parseDate(x, Quarter = True):
    x = str(x)
    #print(x)
    month = x[:2]
    year = x[2:]
    if Quarter == True:
        Q = assignQ(month)
        result = year + Q
    else:
         result = "/".join([year,month])
    return result

In [144]:
#get leaseDate column and apply function parseDate
leaseDate = ura_df[['leaseDate']].applymap(parseDate)
leaseDate.head()

,leaseDate
0,20Q2
1,20Q2
2,20Q2
3,20Q2
4,20Q2


In [145]:
#replace existing leaseDate column
ura_df[['leaseDate']] = leaseDate[['leaseDate']]
ura_df.head()

,areaSqft,areaSqm,district,leaseDate,noOfBedRoom,propertyType,rent,project,street,x,y,psf
0,2350,210-220,15,20Q2,3,Non-landed Properties,4650,ELLIOT AT THE EAST COAST,ELLIOT ROAD,38883.874192089998,32621.50999211,1.97872
1,2350,210-220,15,20Q2,3,Non-landed Properties,5600,ELLIOT AT THE EAST COAST,ELLIOT ROAD,38883.874192089998,32621.50999211,2.38298
2,2550,230-240,15,20Q2,4,Non-landed Properties,6200,ELLIOT AT THE EAST COAST,ELLIOT ROAD,38883.874192089998,32621.50999211,2.43137
3,950,90-100,15,20Q2,2,Non-landed Properties,3100,ELLIOT AT THE EAST COAST,ELLIOT ROAD,38883.874192089998,32621.50999211,3.26316
4,2550,230-240,15,20Q2,4,Non-landed Properties,5700,ELLIOT AT THE EAST COAST,ELLIOT ROAD,38883.874192089998,32621.50999211,2.23529


In [146]:
#make sure each colum have the correct datatype
ura_df.dtypes

areaSqft        object
areaSqm         object
district        object
leaseDate       object
noOfBedRoom     object
propertyType    object
rent             int64
project         object
street          object
x               object
y               object
psf             object
dtype: object

In [147]:
# The code was removed by Watson Studio for sharing.

,street,x,project,y,areaSqft,leaseDate,propertyType,district,noOfBedRoom,rent,areaSqm,psf
0,ELLIOT ROAD,38883.874192,ELLIOT AT THE EAST COAST,32621.509992,950,18Q1,Non-landed Properties,15,2.0,3000,90-100,3.157895
1,ELLIOT ROAD,38883.874192,ELLIOT AT THE EAST COAST,32621.509992,1750,18Q1,Non-landed Properties,15,4.0,5500,160-170,3.142857
2,ELLIOT ROAD,38883.874192,ELLIOT AT THE EAST COAST,32621.509992,2650,18Q1,Non-landed Properties,15,4.0,6000,240-250,2.264151
3,ELLIOT ROAD,38883.874192,ELLIOT AT THE EAST COAST,32621.509992,2150,18Q1,Non-landed Properties,15,3.0,4700,190-200,2.186047
4,ELLIOT ROAD,38883.874192,ELLIOT AT THE EAST COAST,32621.509992,1650,18Q1,Non-landed Properties,15,2.0,2900,140-150,1.757576


In [148]:
dict(sample_ura.dtypes)

{'street': dtype('O'),
 'x': dtype('float64'),
 'project': dtype('O'),
 'y': dtype('float64'),
 'areaSqft': dtype('int64'),
 'leaseDate': dtype('O'),
 'propertyType': dtype('O'),
 'district': dtype('int64'),
 'noOfBedRoom': dtype('float64'),
 'rent': dtype('int64'),
 'areaSqm': dtype('O'),
 'psf': dtype('float64')}

In [149]:
ura_df = ura_df.astype(dtype = dict(sample_ura.dtypes))
ura_df.dtypes

areaSqft          int64
areaSqm          object
district          int64
leaseDate        object
noOfBedRoom     float64
propertyType     object
rent              int64
project          object
street           object
x               float64
y               float64
psf             float64
dtype: object

In [150]:
# The code was removed by Watson Studio for sharing.

In [151]:
#append ura_df to existing URA_RENTAL sql table

ura_df.to_sql('ura_rental', engine, if_exists = 'append', index = False)

In [152]:
#get projects and districts from URA_RENTAL
ura_project = engine.execute('SELECT PROJECT,DISTRICT,"propertyType" FROM "URA_RENTAL"')
ura_project = pd.DataFrame(ura_project)
ura_project.head()

,0,1,2
0,ELLIOT AT THE EAST COAST,15,Non-landed Properties
1,ELLIOT AT THE EAST COAST,15,Non-landed Properties
2,ELLIOT AT THE EAST COAST,15,Non-landed Properties
3,ELLIOT AT THE EAST COAST,15,Non-landed Properties
4,ELLIOT AT THE EAST COAST,15,Non-landed Properties


In [153]:
ura_project = ura_project.drop_duplicates()
ura_project.columns = ['PROJECT','DISTRICT','propertyType']
ura_project.head()

,PROJECT,DISTRICT,propertyType
0,ELLIOT AT THE EAST COAST,15,Non-landed Properties
9,MERAWOODS,23,Non-landed Properties
18,THOMSON RISE ESTATE,20,Detached House
19,THE ESPIRA,15,Non-landed Properties
21,GRAY MANSIONS,15,Non-landed Properties


In [154]:
#load ura_projects to sql table
ura_project.to_sql('ura_project', engine, if_exists = 'replace', index = False)

In [155]:
engine.dispose()

In [156]:
# The code was removed by Watson Studio for sharing.

2020-08-14 08:12:52,054 - __PROJECT_LIB__ - ERROR - failed to initialize ibmos2spark integration
Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/project_lib/storage/bcos.py", line 138, in _initialize_bcos2spark
    import ibmos2spark
ModuleNotFoundError: No module named 'ibmos2spark'


,quarter
0,19Q3
1,19Q4
2,20Q1
3,20Q2
